# Présentation des données

In [ ]:
# Exo 1 - Préparation des données 

# importation des données
import pandas as pd 
import numpy as np 
# import du fichier
data = pd.read_csv("synthetic.csv")

# Visualisation des données
print(data.head())


In [ ]:
# 1 - Nombre de colonnes (attributs) dans le DataFrame
num_attributes = data.shape[1]

# Afficher le nombre d'attributs
print(f"Le nombre d'attributs dans le fichier est : {num_attributes}")


In [ ]:
# Type de données et valeurs manquantes
print(data.info())

In [ ]:
# Avoir le nombre d'attributs dans le modèle
print(data.columns)
# 14 attributs dans le modèle

In [ ]:
# Obtenir les classes uniques dans la colonne 'Class'
classes_uniques = data['Class'].unique()

# Nombre de classes différentes
num_classes = len(classes_uniques)

# Afficher le nombre de classes différentes
print(f"Le nombre de classes différentes dans les données est : {num_classes}")

In [ ]:
# combien d'instances compte chaque classe?
nbr_instances = data['Class'].value_counts()
print(nbr_instances)

# Sortie 
# Class
# 1    908
# 0    674
# 2    472
# 3    244
# Name: count, dtype: int64

# Les données sont-elles linéairement séparables ?
Non, si on observe le schéma 1 on voit que les données ne le sont pas.
De plus si l'on choisit de les ranger par classe , on peut s'apercevoir que 


In [ ]:
import matplotlib.pyplot as plt # import biblio matplot
plt.figure(figsize=(10, 6))
plt.scatter(data['Attr_A'], data['Attr_B'], c=data['Class'], alpha=0.5, cmap='viridis')
plt.xlabel('Attribut 1')
plt.ylabel('Attribut 2')
plt.title('Scatter Plot des attributs par classe')
plt.colorbar(label='Classe')
plt.show()

# On peut voir clairement que ce n'est pas divisible linéairement à l'état brut
# je pense que use image est vraiment mieux


## 5 et 6 (voir compte-rendu.md) 

# 2 Mise en oeuvre des modèles

In [ ]:
# Choisir un attribut à analyser, par exemple 'Attr_A'
attribute = 'Attr_A'


# Calculer les quartiles pour l'attribut choisi
quartiles = data[attribute].quantile([0.25, 0.5, 0.75])

# Sort the attribute values and print them
sorted_attribute = data[attribute].sort_values()
print(sorted_attribute)
print(quartiles)
# Afficher les quartiles
print(f"Quartile 1 (Q1) de l'attribut '{attribute}': {quartiles[0.25]}")
print(f"Médiane (Q2) de l'attribut '{attribute}': {quartiles[0.5]}")
print(f"Quartile 3 (Q3) de l'attribut '{attribute}': {quartiles[0.75]}")


# 6 : 

# Partie 2 : Mise en oeuvre des modèles.

# Arbre de décision 


In [ ]:
# Arbre de décision

# Calcul de l'entropie

"""
L'entropie est une mesure de l'incertitude associée à une variable aléatoire.
"""

def entropie(dataframe , attribut_cible):  
    # Calcul de la probabilité de chaque classe
    compte_classe = dataframe[attribut_cible].value_counts()
    #print(compte_classe)
    proba = compte_classe / compte_classe.sum()
    #print(proba) 
    # Calcul de l'entropie
    entropie = - (proba * np.log2(proba+ np.finfo(float).eps)).sum() # éviter log2(0)
    return entropie

# Test de la fonction
print(entropie(data, 'Attr_A'))


11.166163082646115
11.166163082645376

11.166163082646115
11.166163082645376

11.166163082646115
11.166163082645376

1.8608867211835993
1.860886721183598

In [ ]:
import pandas as pd
import numpy as np

# Fonction pour calculer tous les quartiles d'un attribut donné
def calculate_quartiles(data, attribute):
    return data[attribute].quantile([0.25, 0.5, 0.75])

# Test de la fonction sur le DataFrame chargé

print(calculate_quartiles(data, 'Attr_A'))


In [ ]:
data

In [ ]:
data.sort_values(by="Attr_C")

In [ ]:
# data.head()
sorted = data.sort_values(by="Attr_A")
print(len(sorted))

In [ ]:
# Fonction qui calcule le gain d'une partition
def gain_information(dataframe, attribut_cible, attribut_test):
    # Initial entropy of the target attribute
    entropie_initiale = entropie(dataframe, attribut_cible)

    # The gain, split_value and partitions initialized
    max_gain = 0
    best_split_value = None
    best_partitions = None

    # Sorting data by the attribute to test
    sorted_data = dataframe.sort_values(by=attribut_test)

    # Unique values of the attribute to test, considering quartiles to reduce complexity
    unique_values = calculate_quartiles(sorted_data, attribut_test).to_list()
    print("unique values", unique_values);

    # Adding the min and max values to cover the entire range of the attribute
    unique_values = [sorted_data[attribut_test].min()] + \
        unique_values + [sorted_data[attribut_test].max()]
    
    print(f"unique values after {unique_values}");

    # Iterating through the sorted unique values to find the best split
    for split_value in unique_values:
        # Partitioning the data based on the split value
        lower_partition = sorted_data[sorted_data[attribut_test] < split_value]
        upper_partition = sorted_data[sorted_data[attribut_test]
                                      >= split_value]

        # Calculating the weighted entropy for the partitions
        # Row counts.
        total_instances = len(sorted_data)
        lower_weight = len(lower_partition) / total_instances
        upper_weight = len(upper_partition) / total_instances

        # Computing the weighted_entropy 
        weighted_entropy = (lower_weight * entropie(lower_partition, attribut_cible)) + \
                           (upper_weight * entropie(upper_partition, attribut_cible))

        # Information gain for the current split
        current_gain = entropie_initiale - weighted_entropy

        # If the current gain is greater than the max_gain, update max_gain and best_split_value
        if current_gain > max_gain:
            max_gain = current_gain
            best_split_value = split_value
            best_partitions = (lower_partition, upper_partition)

    # Returning the attribute, gain, split_value, and partitions as a tuple
    return attribut_test, max_gain, best_split_value, best_partitions


# Testing the function with an example attribute
# Let's use 'Attr_A' as the attribute to test and 'Class' as the target
test_gain_info = gain_information(data, 'Class', 'Attr_C')
test_gain_info

In [ ]:
# Fonction qui reçoit un dataframe et retourne
def func(attribut):
    data_sorted = data.sort_values(by= attribut) 